# Rule 1 Key planning documents not provided

In [1]:
# import libraries
import json
import pandas as pd
import matplotlib.pyplot as plt
import folium
import gc
from pandas import DataFrame

In [2]:
#Read original CSV files to extract required data fields (procurement method, amount, buyer, region, cycle)
l = [i for i in range(1,14)]
for n in l:
    s="c"+str(n)+".csv"
    data=pd.read_csv(s)
    contratos=data[['releases']]
    
    a=[]#list of contract amounts
    b=[]#list of buyers
    c=[]#list of regions
    d=[]#List of cycles
    e=[]#List of planningStatus
    
    #Read json to extract data fields in contratos
    for n in contratos.iterrows():
        cifra=json.loads(n[1]["releases"])
        r=len(cifra)-1
        amount=0
        buyer=""
        region=""
        cycle=""
        planningStatus="Yes"
        
        #Get cycle
        try:
            cycle=cifra[r]["cycle"] 
        except:
            cycle=cifra[r]["ciclo"]
        
        #Get sum of amounts
        try:
            for m in cifra[r]["contracts"]:
                amount=amount+(m["value"]["amount"])
        except:
            amount=0
        
        for m in cifra[r]["parties"]:
            
            #Get buyer
            try:
                if m["roles"][0]=="buyer":
                    buyer=m["name"]
            except:
                     buyer=""                
            
            #Get region
            try:
                if m["roles"][0]=="procuringEntity":
                    region=m["address"]["region"]
            except:
                region=""

        #Get planningStatus
        try:
            planningsss=cifra[r]["planning"] 
        except:
            planningStatus="No"
            
        a.append(amount)
        b.append(buyer)
        c.append(region)
        d.append(cycle)
        e.append(planningStatus)
        
    tenderers=data[["ocid","compiledRelease.tender.procurementMethod",'compiledRelease.buyer.name','compiledRelease.tender.value.amount']]
    tenderers['amount0'] = a
    tenderers['procuringEntity'] = b
    tenderers['region'] = c
    tenderers['cycle'] = d
    tenderers['planningStatus'] = e
    
    #Appending result to a new CSV
    #Remove , and rowsIDs later
    tenderers.to_csv("planning01.csv", header=False, mode = 'a',index=False)
    
# Destroy unused objects
del a,b,c,d,e,amount,buyer,region,cycle,tenderers,planningStatus,contratos,data
# Run garbage collection process 
gc.collect()

C:\Users\Dagoberto\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Dagoberto\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Dagoberto\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing

26

In [5]:
#Read new file
data=pd.read_csv("planning0.csv", encoding ='latin1', header=None,skiprows=[0])
data=data.rename(columns={0:'ocid', 1:'procurementMethod', 2:'buyer1', 3:'amount0', 4:'amount1', 5:'buyer2', 6:'region', 7:'cycle', 8:'planningStatus'})

#Define function to identify finalAmount
def f(row):
    if row['amount0'] > row['amount1']:
        val = row['amount0']
    else:
        val = row['amount1']
    return val

#Apply function to dataframe
data['finalAmount'] = data.apply(f, axis=1)

#Label unknown procurement methods
#mask = data.procurementMethod == "0"
#column_name = 'procurementMethod'
data.loc[data.procurementMethod == "0", 'procurementMethod'] = "unknown"
data.loc[data.buyer2 == "0", 'buyer2'] = "unknown"
data.loc[data.region == "0", 'region'] = "unknown"

In [6]:
data.head()

,ocid,procurementMethod,buyer1,amount0,amount1,buyer2,region,cycle,planningStatus,finalAmount
0,ocds-07smqs-1072,NaN,NaN,NaN,0.0,delegación SEDESOL en Tabasco,NaN,2017,Yes,0.0
1,ocds-07smqs-1111,NaN,NaN,NaN,0.0,delegación SEDESOL en México,NaN,2017,Yes,0.0
2,ocds-07smqs-1117,NaN,NaN,NaN,0.0,Instituto Nacional de Antropología e Historia,NaN,2017,Yes,0.0
3,ocds-07smqs-1182,NaN,NaN,NaN,0.0,Servicio de Administración Tributaria,NaN,2017,Yes,0.0
4,ocds-07smqs-1191,NaN,NaN,NaN,0.0,delegación SEDESOL en Tabasco,NaN,2017,Yes,0.0
